In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchtext
from torchtext import data, datasets
from torchtext.vocab import Vocab

import time


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [7]:
torch.cuda.is_available()

False

In [2]:
from torchtext.datasets import IMDB

train_iter, test_iter = IMDB(split=('train' , 'test'), root = './data')

In [3]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')

In [4]:
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

def get_vocab(train_datapipe):
    vocab = build_vocab_from_iterator(yield_tokens(train_datapipe), specials=['<UNK>', '<PAD>','<BOS>', '<EOS>'], max_tokens=20000)
    vocab.set_default_index(vocab['<UNK>'])
    return vocab

vocab = get_vocab(train_iter)

In [5]:
print("The length of the new vocab is", len(vocab))
new_stoi = vocab.get_stoi()
print("The index of '' is", new_stoi['<PAD>'])
new_itos = vocab.get_itos()
print("The token at index 2 is", new_itos[2])
     

The length of the new vocab is 20000
The index of '' is 1
The token at index 2 is <BOS>


In [6]:
text_transform = lambda x: [vocab['<BOS>']] + [vocab[token] for token in tokenizer(x)] + [vocab['<EOS>']]
label_transform = lambda x: 1 if x == 'pos' else 0

In [8]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    label_list, text_list = [], []

    for (_label, _text) in batch:
        label_list.append(label_transform(_label))
        processed_text = torch.tensor(text_transform(_text))
        text_list.append(processed_text)

    return torch.tensor(label_list), pad_sequence(text_list, padding_value=3.0)

train_dataloader = DataLoader(list(train_iter), batch_size=64, shuffle=True, collate_fn=collate_batch)

In [13]:
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embedding_dim = 100
hidden_size = 300

In [14]:
print(device)

cpu


In [12]:
torch.__version__

'1.13.1+cpu'